In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import ShuffleSplit as spliter
from sklearn.metrics import mean_squared_error as mse
import lightgbm as lgb
from tqdm import tqdm
import warnings, gc, sys, os

warnings.filterwarnings('ignore')
sys.path.append(os.path.abspath(os.path.dirname("../")))
import hn_elo as elo

In [10]:
path = '../../dataset/'

train = pd.read_hdf(path+'trainPrep1.h5')
trainPure = pd.read_hdf(path+'pureTrain.h5')
test = pd.read_hdf(path+'testPrep1.h5')

hist = pd.read_hdf(path+'histPrep2.h5')
new = pd.read_hdf(path+'newPrep2.h5')

# mer = pd.read_csv(path+'merchants.csv')

In [7]:
hist.to_hdf(path+'histPrep1.h5', 'histPrep1')
new.to_hdf(path+'newPrep1.h5', 'newPrep1')
# train.to_hdf(path+'trainPrep1.h5', 'trainPrep1')
# test.to_hdf(path+'testPrep1.h5', 'testPrep1')
# trainPure.to_hdf(path+'pureTrain.h5', 'pureTrain')

In [12]:
# create agg functions
from scipy import stats
def mode(x):
    return stats.mode(x)[0][0]

merged = {
    'card_id':'count',
    'authorized_flag' : 'mean',
    'city_id' : [mode, 'nunique'],
    'category_1': [mode, 'mean', 'median', 'std', 'max','min'],
    'installments' : ['mean', 'median', 'std', 'max','min'],
    'category_3' : [mode, 'mean', 'median', 'std', 'max', 'min'],
    'merchant_category_id' : [mode,'nunique'],
    'merchant_id' : [mode, 'nunique'],
    'month_lag':['mean', 'median', 'std', 'max', 'min'],
    'purchase_amount':['mean', 'median', 'std', 'max', 'min'],
    'fromRefDate':['min','max','mean','median','std'],
    'purchaseYear':['mean', 'median', 'std'],
    'purchaseMonth':['mean', 'median', 'std'],
    'purchaseDay':['mean', 'median', 'std'],
    'purchaseHour':['mean', 'median', 'std'],
    'category_2' : [mode, 'mean', 'median', 'std','max','min'],
    'state_id':[mode,'nunique'],
    'subsector_id':[mode, 'nunique'],
}

# zip hist and new
trans = hist.append(new)
trans = trans.reset_index()
transByCard = trans.groupby('card_id').agg(merged)
transByCard = transByCard.reset_index()

# merging trans + train/test
trainMerged = pd.merge(train, transByCard, how='left', on='card_id')
testMerged = pd.merge(test, transByCard, how='left', on='card_id')
trainPureMerged = pd.merge(trainPure, transByCard, how='left', on='card_id')

In [20]:
# create colum names and category column names using for train
modelCols = elo.modelCols(trainMerged, 
                          removal = ('target', 'card_id','first_active',))
def createCats(removal=None):
    ret = [
            'feature_1','feature_2','feature_3',
             "('city_id', 'mode')","('category_1', 'mode')",
             "('category_3', 'mode')","('merchant_category_id', 'mode')",
             "('category_2', 'mode')","('state_id', 'mode')",
             "('subsector_id', 'mode')","('merchant_id', 'mode')",
            ]
    if removal is not None :
        for rm in removal:
            ret.remove(rm)
    return ret
modelCats = createCats()

# parameters
paramsValidate = {
    'objective':'regression',
    'metric':'l2',
    'num_threads':8,
}

splitRate = test.shape[0]/(train.shape[0]+test.shape[0])
folder = spliter(5, splitRate)

In [15]:
# reanme
trainMerged = elo.renameCols(trainMerged, modelCols)
testMerged = elo.renameCols(testMerged, modelCols)
trainPureMerged = elo.renameCols(trainPureMerged, modelCols)

In [25]:
# train 5-fold quick lgb
basicImprts, basicScore = elo.quickLgbValidate(
    trainMerged, folder, modelCols, modelCats, paramsValidate)

3it [00:06,  2.03s/it]


In [27]:
basicScore.mean()

test     2.795520
train    2.403949
dtype: float64

In [36]:
modelCols = basicImprts[basicImprts['mean']>100].index.tolist()
modelCats = list(set(modelCats)&set(modelCols))

basicCoreImprts, basicCoreScore = quickLgbValidate(
    trainMerged, folder, modelCols, modelCats, paramsValidate)

5it [00:04,  1.12it/s]


colName
feature_1                           5.331073e+02
feature_2                           2.921668e+02
feature_3                           2.500764e+02
first_active_year                   1.946795e+02
first_active_month                  1.051816e+03
lowerBound                          7.871493e+06
('authorized_flag', 'mean')         4.498616e+03
('city_id', 'mode')                 1.045675e+05
('category_1', 'mean')              7.387484e+02
('category_1', 'std')               8.714387e+02
('installments', 'mean')            2.172225e+03
('installments', 'median')          2.263768e+02
('installments', 'std')             8.653832e+02
('category_3', 'mode')              1.793574e+02
('category_3', 'mean')              2.570565e+03
('category_3', 'std')               9.691240e+02
('merchant_category_id', 'mode')    2.021216e+04
('month_lag', 'mean')               3.262462e+04
('month_lag', 'median')             6.318819e+03
('month_lag', 'std')                3.093077e+04
('purchase_a

## except target-min

In [17]:
modelCols = elo.modelCols(trainPureMerged, 
                          removal=('target','card_id','lowerBound','first_active'))
modelCats = createCats()
pureImprts, pureScore = quickLgbValidate(
    trainPureMerged, folder, modelCols, modelCats, paramsValidate)

5it [00:04,  1.02it/s]


In [25]:
modelCols = importancesPure[importancesPure['mean']>100].index.tolist()
modelCats = list(set(modelCats)&set(modelCols))

pureCoreImprts, pureCoreScore = quickLgbValidate(
    trainPureMerged, folder, modelCols, modelCats, paramsValidate)

5it [00:04,  1.15it/s]


colName
feature_1                              326.448401
feature_2                              159.455278
feature_3                              223.163220
first_active_year                      187.023301
first_active_month                     972.588816
('authorized_flag', 'mean')           4377.450773
('city_id', 'mode')                 106615.325029
('category_1', 'mean')                 634.061381
('category_1', 'std')                  812.112837
('installments', 'mean')              1824.592340
('installments', 'median')             280.114799
('installments', 'std')               1027.715221
('category_3', 'mode')                  94.482661
('category_3', 'mean')                2637.280474
('category_3', 'std')                 1053.045339
('merchant_category_id', 'mode')     20557.558931
('month_lag', 'mean')                33288.405067
('month_lag', 'median')               4607.837008
('month_lag', 'std')                 31939.982106
('purchase_amount', 'mean')           1462

## Base Score

In [12]:
modelCols = elo.modelCols(trainPureMerged, 
                          removal=('target', 'card_id', 'first_active','lowerBound'))
modelCats = createCats()
submission, bst = quickLgbTest(trainPuremerged, testMerged, modelCols, modelCats, params)

In [17]:
submission.to_csv('20190124-submission-quick-lgb.csv', index=False)